In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates

import time
# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory

from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

Constants

In [2]:
PATH = "C:/Users/davez/Desktop/Investing with Data Science/"
S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)


Get Column Data from CSV

In [3]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesnt exist issue a warning
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name]

Get Stock Tickers

In [4]:
tickers = get_column_from_csv("C:/Users/davez/Desktop/Investing with Data Science/Wilshire-5000-Stocks-New.csv", "Ticker")
print(len(tickers))
print(tickers)

3481
0          A
1         AA
2        AAL
3       AAME
4        AAN
        ... 
3476    ZUMZ
3477     ZUO
3478    ZYNE
3479    ZYXI
3480    ZNGA
Name: Ticker, Length: 3481, dtype: object


Get Dataframe from CSV

In [5]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_stock_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

In [6]:
print(tickers)

0          A
1         AA
2        AAL
3       AAME
4        AAN
        ... 
3476    ZUMZ
3477     ZUO
3478    ZYNE
3479    ZYXI
3480    ZNGA
Name: Ticker, Length: 3481, dtype: object


In [7]:

files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers
# tickers.remove('.ds_Store') MacOS Only
tickers.sort()
len(tickers)

3263

In [8]:
# Add Daily Returns
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    #df.to_csv(PATH + ticker + '.csv')
    return df

In [9]:
# Add Cumulative Returns
def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    #df.to_csv(PATH + ticker + '.csv')
    return df

In [10]:
# Add Bollinger Bands
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    return df
    # df.to_csv(PATH + ticker + '.csv')

In [11]:
# Add Ichimoku Data to Dataframe
def add_Ichimoku(df):
    # Conversion Line = (Highest value in period + Lowest value in period)/2 (9 sessions)
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2
    
    # Base Line = (Highest value in period + Lowest value in period)/2 (26 sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2
    
    # Span A = (Conversion Value + Base Value)/2 (26 sessions)
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)
    
    # Span B = (Conversion Value + Base Value)/2 (52 sessions)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)
    
    # Lagging Span = Price shifted back 26 periods
    df['Lagging'] = df['Close'].shift(-26)
    return df
    # df.to_csv(PATH + ticker + '.csv')

In [12]:
# Always a good idea to test using one stock first (rather than 3500, which takes a little while)...
# try on ticker A
try:
    print("Working on :", "A")
    new_df = get_stock_df_from_csv("A")
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_Ichimoku(new_df)
    new_df.to_csv(PATH + 'A' + '.csv')
except Exception as ex:
    print(ex)

Working on : A


In [13]:
for x in tickers:
    try:
        print("Working on :", x)
        new_df = get_stock_df_from_csv(x)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bollinger_bands(new_df)
        new_df = add_Ichimoku(new_df)
        new_df.to_csv(PATH + x + '.csv')
    except Exception as ex:
        print(ex)

Working on : A
Working on : AA
Working on : AAL
Working on : AAME
Working on : AAN
Working on : AAOI
Working on : AAON
Working on : AAP
Working on : AAPL
Working on : AAT
Working on : AAWW
Working on : AAXN
Working on : ABBV
Working on : ABC
Working on : ABCB
Working on : ABEO
Working on : ABG
Working on : ABIO
Working on : ABM
Working on : ABMD
Working on : ABR
Working on : ABT
Working on : ABTX
Working on : AC
Working on : ACA
Working on : ACAD
Working on : ACBI
Working on : ACC
Working on : ACCO
Working on : ACER
Working on : ACGL
Working on : ACHC
Working on : ACHV
Working on : ACIA
Working on : ACIW
Working on : ACLS
Working on : ACM
Working on : ACMR
Working on : ACN
Working on : ACNB
Working on : ACOR
Working on : ACRE
Working on : ACRS
Working on : ACRX
Working on : ACTG
Working on : ACU
Working on : ACY
Working on : ADBE
Working on : ADC
Working on : ADES
Working on : ADI
Working on : ADM
Working on : ADMA
Working on : ADMP
Working on : ADMS
Working on : ADNT
Working on : ADP


Plot Bollinger Bands

In [14]:
def plot_with_boll_bands(df, ticker):
    fig = go.Figure()
    
    candle = go.Candlestick(x=df.index, open=df['Open'],
    high=df['High'], low=df['Low'],
    close=df['Close'], name='Candlestick')
    
    upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
    line=dict(color='rgba(250, 0, 0, 0.75)', 
    width=1), name="Upper Band")
    
    mid_line = go.Scatter(x=df.index, y=df['middle_band'],
    line=dict(color='rgba(0, 0, 250, 0.75)',
    width=0.7), name='Middle Band') 
    
    lower_line = go.Scatter(x=df.index, y=df['lower_band'],
    line=dict(color='rgba(0, 250, 0, 0.75)',
    width=1), name='Lower Band') 
    
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)
    
    fig.update_xaxes(title='Date', rangeslider_visible=True)
    fig.update_yaxes(title='Price')
    
    fig.update_layout(title=ticker + "Bollinger Bands",
    height=900, width=800, showlegend=True)
    fig.show()

In [15]:
test_df = get_stock_df_from_csv('AMD')
test_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging
Date,,,,,,,,,,,,,,,,,
2017-02-06,12.46,13.70,12.38,13.63,140474100,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.98
2017-02-07,14.05,14.27,13.06,13.29,158683800,0,0,-0.024945,0.975055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.65
2017-02-08,13.21,13.75,13.08,13.56,75942900,0,0,0.020316,0.994864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.49
2017-02-09,13.78,13.89,13.40,13.42,73339900,0,0,-0.010325,0.984593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.40
2017-02-10,13.86,13.86,13.25,13.58,54579300,0,0,0.011922,0.996332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.82


Plot Ichimoku

In [16]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0, 250, 0, 0.4)'
    else:
        return 'rgba(250, 0, 0, 0.4)'

Function to create Ichimoku plot

In [17]:
def get_Ichimoku(df):
    candle = candle = go.Candlestick(x=df.index, open=df['Open'],
    high=df['High'], low=df['Low'],
    close=df['Close'], name='Candlestick')
    
    # this block defines the fills between the spans
    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum() # .ne() = not equal to
    df = df.groupby('group')  
    
    dfs = []
    for name, data in df:
        dfs.append(data)
        
    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y=df.SpanA, 
        line=dict(color='rgba(0,0,0,0)')))
        
        fig.add_traces(go.Scatter(x=df.index, y=df.SpanB,
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=get_fill_color(df['label'].iloc[0])))
    
    baseline = go.Scatter(x=df1.index, y=df1['Baseline'],
    line=dict(color='pink', width=2), name='Baseline')
    
    conversion = go.Scatter(x=df1.index, y=df1['Conversion'],
    line=dict(color='black', width=1), name='Conversion')
    
    lagging = go.Scatter(x=df1.index, y=df1['Lagging'],
    line=dict(color='purple', width=2), name='Lagging')
    
    span_a = go.Scatter(x=df1.index, y=df1['SpanA'],
    line=dict(color='green', width=2, dash='dot'), name='Span A')
    
    span_b = go.Scatter(x=df1.index, y=df1['SpanB'],
    line=dict(color='red', width=1, dash='dot'), name='Span B')
    
    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    fig.update_layout(height=900, width=800, showlegend=True)
    fig.show()

In [18]:
    test_df = get_stock_df_from_csv('AMD')
#plot_with_boll_bands(test_df, 'AAOI')
get_Ichimoku(test_df)

Portfolio Analyis

In [19]:
# Get Sector Stocks

sec_df = pd.read_csv("C:/Users/davez/Desktop/Investing with Data Science/big_stock_sectors.csv")

indus_df = sec_df.loc[sec_df['Sector'] == 'Industrial']
health_df = sec_df.loc[sec_df['Sector'] == 'Healthcare']
it_df = sec_df.loc[sec_df['Sector'] == 'Information Technology']
comm_df = sec_df.loc[sec_df['Sector'] == 'Communication']
staples_df = sec_df.loc[sec_df['Sector'] == 'Staples']
discretion_df = sec_df.loc[sec_df['Sector'] == 'Discretionary']
utility_df = sec_df.loc[sec_df['Sector'] == 'Utilities']
financial_df = sec_df.loc[sec_df['Sector'] == 'Financials']
material_df = sec_df.loc[sec_df['Sector'] == 'Materials']
restate_df = sec_df.loc[sec_df['Sector'] == 'Real Estate']
energy_df = sec_df.loc[sec_df['Sector'] == 'Energy']

In [20]:
def get_cum_ret_for_stocks(stock_df):
    tickers = []
    cum_rets = []

    for index, row in stock_df.iterrows():
        df = get_stock_df_from_csv(row['Ticker'])
        if df is None:
            pass
        else:
            tickers.append(row['Ticker'])
            cum = df['cum_return'].iloc[0]
            cum_rets.append(cum)
    return pd.DataFrame({'Ticker':tickers, 'CUM_RET':cum_rets})

In [21]:
industrial = get_cum_ret_for_stocks(indus_df)
industrial

File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't

,Ticker,CUM_RET
0,AAL,NaN
1,AAOI,NaN
2,AAON,NaN
3,AAWW,NaN
4,ABM,NaN
...,...,...
344,WSC,NaN
345,WSTG,NaN
346,WTT,NaN
347,XPO,NaN
